In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import json
from tqdm import tqdm
from utils import *
from bs4 import BeautifulSoup
from collections import Counter
from feature_settings import *

Security prices


In [2]:
# security prices
security_prices_df = pd.read_csv(r"raw_data\OpitonMetrics\securities_data\security_prices.csv")

for i in security_prices_df.keys():
    if (
        i != 'date' and 
        i != 'low' and 
        i != 'high' and
        i != 'open' and 
        i != 'close' and 
        i != 'return' and 
        i != 'class' and 
        i != 'industry_group'
    ):
        print(f"The {i} is only one value: {set(security_prices_df[i])}")
print('\n')

print(f"The class is all NaN: {all(pd.isna(i) for i in security_prices_df['class'])}")
print(f"The industry_group is all NaN: {all(pd.isna(i) for i in security_prices_df['industry_group'])}")
print(f"The high price is not necessarily greater than close price: {any(security_prices_df['high'] < security_prices_df['close'])}")

The secid is only one value: {108105}
The cusip is only one value: {64881510}
The ticker is only one value: {'SPX'}
The sic is only one value: {9999}
The index_flag is only one value: {1}
The exchange_d is only one value: {32768}
The issue_type is only one value: {'A'}
The volume is only one value: {0}
The cfadj is only one value: {1}
The shrout is only one value: {0}
The cfret is only one value: {1}


The class is all NaN: True
The industry_group is all NaN: True
The high price is not necessarily greater than close price: True


historical volatility

In [3]:
# historical_vol
historical_vol_df = pd.read_csv(r"raw_data\OpitonMetrics\options_data\historical_vol.csv")

# check the cols
for i in historical_vol_df.keys():
    if (
        i != 'date' and
        i != 'volatility' and
        i != 'industry_group' and 
        i != 'class'
    ):
        print(f"The {i} is only the value: {set(historical_vol_df[i])}")
print('\n')
print(f"The industry_group is all NaN: {all(pd.isna(i) for i in historical_vol_df['industry_group'])}")
print(f"The class is all NaN: {all(pd.isna(i) for i in historical_vol_df['class'])}")

# pivot the table
historical_vol_df_pivot = historical_vol_df.pivot(index='date', columns='days', values='volatility')
historical_vol_df_pivot.columns = [f'vol_days_{i}' for i in historical_vol_df_pivot.columns]
print('\n')
# check after pivot, there is no missing value
print(f"There is no missing value after pivot: {not historical_vol_df_pivot.isna().any().any()}")

The secid is only the value: {108105}
The days is only the value: {1825, 547, 10, 365, 14, 273, 182, 152, 730, 122, 91, 60, 30}
The cusip is only the value: {64881510}
The ticker is only the value: {'SPX'}
The sic is only the value: {9999}
The index_flag is only the value: {1}
The exchange_d is only the value: {32768}
The issue_type is only the value: {'A'}


The industry_group is all NaN: True
The class is all NaN: True


There is no missing value after pivot: True


Forward prices

In [4]:
# forward prices
forward_price_df = pd.read_csv(r"raw_data\OpitonMetrics\options_data\forward_price.csv")

print(f"Forward price columns are: {forward_price_df.keys()}")
print('\n')
# check if there is any missing value
print(f"There is no missing value in the forward price data: {not forward_price_df.isna().any().any()}")


Forward price columns are: Index(['secid', 'date', 'expiration', 'AMSettlement', 'ForwardPrice', 'cusip',
       'ticker', 'sic', 'index_flag', 'exchange_d', 'class', 'issue_type',
       'industry_group', 'issuer'],
      dtype='object')


There is no missing value in the forward price data: False


Fama French 5 factors

In [5]:
# load the data
fama_5_factors_df = pd.read_csv(r"raw_data\FamaFrench\5_factors.csv")

# check if there is any missing value
print(f"There is no missing value in the Fama French 5 factors data file: {not fama_5_factors_df.isna().any().any()}")

There is no missing value in the Fama French 5 factors data file: True


index dividend yield

In [6]:
# load the data
dividend_rate_df = pd.read_csv(r"raw_data\OpitonMetrics\market_data\index_dividend_yield.csv")

# check all the cols
for col in dividend_rate_df.columns:
    if not (
        col == 'date' or 
        col == 'expiration' or
        col == 'class' or 
        col == 'industry_group' or 
        col == 'rate'
    ):
        print(f"The {col} is only one value: {set(dividend_rate_df[col])}")
print('\n')
print(f"The class is only NaN: {all(pd.isna(i) for i in dividend_rate_df['class'])}")
print(f"The industry_group is only NaN: {all(pd.isna(i) for i in dividend_rate_df['industry_group'])}")
print(f"The useful data have no missing value: {not dividend_rate_df[['date', 'expiration', 'rate']].isna().any().any()}")

The secid is only one value: {108105}
The cusip is only one value: {64881510}
The ticker is only one value: {'SPX'}
The sic is only one value: {9999}
The index_flag is only one value: {1}
The exchange_d is only one value: {32768}
The issue_type is only one value: {'A'}


The class is only NaN: True
The industry_group is only NaN: True
The useful data have no missing value: True


volume and open interest

In [7]:
# load the data
volume_open_interest_df = pd.read_csv(r"raw_data\OpitonMetrics\options_data\volume_open_interest.csv")

# check if the volume and open interest are all positive or not
print(f"Volume are all positive: {all(i > 0 for i in volume_open_interest_df['volume'])}")
print(f"Open interest are all positive: {all(i > 0 for i in volume_open_interest_df['open_interest'])}")

# check if the volume is always bigger than the open interest
print(f"Volume is always bigger than open interest: {all(volume_open_interest_df['volume'] > volume_open_interest_df['open_interest'])}")

volume_open_interest_df[volume_open_interest_df['open_interest'] <= 0]

Volume are all positive: True
Open interest are all positive: False
Volume is always bigger than open interest: False


,secid,date,cp_flag,index_flag,cusip,ticker,sic,exchange_d,class,issue_type,industry_group,volume,open_interest
2301,108105,1999-01-19,NaN,1,64881510,SPX,9999,32768,NaN,A,NaN,76558,0
2302,108105,1999-01-19,C,1,64881510,SPX,9999,32768,NaN,A,NaN,42521,0
2303,108105,1999-01-19,P,1,64881510,SPX,9999,32768,NaN,A,NaN,34037,0
3561,108105,2000-09-15,NaN,1,64881510,SPX,9999,32768,NaN,A,NaN,118334,0
3562,108105,2000-09-15,C,1,64881510,SPX,9999,32768,NaN,A,NaN,70103,0
3563,108105,2000-09-15,P,1,64881510,SPX,9999,32768,NaN,A,NaN,48231,0
3564,108105,2000-09-18,NaN,1,64881510,SPX,9999,32768,NaN,A,NaN,60080,0
3565,108105,2000-09-18,C,1,64881510,SPX,9999,32768,NaN,A,NaN,18310,0
3566,108105,2000-09-18,P,1,64881510,SPX,9999,32768,NaN,A,NaN,41770,0


features

In [8]:
feature_summation = (
    FEATURES_YEAR_MONTH_DAY +
    FEATURES_GREEKS + 
    FEATURES_IMPLIED_VOL + 
    FEATRUES_BASIC_OPTION +
    FEATURES_SETTLEMENT_INFO + 
    FEATURES_VOLUME_OPEN_INTEREST +
    FEATURES_INTEREST_RATE + 
    FEATURES_FORWARD_PRICE + 
    FEATURES_STOCK + 
    FEATURES_VIX + 
    FEATURES_HISTORICAL_VOL + 
    FEATURES_FAMA + 
    FEATURES_DATE + 
    FEATURES_OTHER + 
    FEATURES_RETURN
)

print(len(feature_summation), len(FEATURES_WHOLE), len(set(feature_summation)), len(set(FEATURES_WHOLE)))
print(set(feature_summation) == set(FEATURES_WHOLE))

98 98 98 98
True


whole_df

In [9]:
# load the dataset
whole_df = pd.read_csv(r"intermediate_files\whole_df.csv")

C:\Users\55479\AppData\Local\Temp\ipykernel_13572\2611208492.py:2: DtypeWarning: Columns (24,25) have mixed types. Specify dtype option on import or set low_memory=False.
  whole_df = pd.read_csv(r"intermediate_files\whole_df.csv")


In [10]:
print(set(whole_df.keys()) == set(FEATURES_WHOLE))

True
